**Hindi TTS Interactive notebook using Tacotron 2 and Waveglow**

In this notebook, we will be using our checkpoint file of the training notebook we ran previously and use the WaveGlow decoder to get the final output of Hindi Text to Speech.

WaveGlow is a flow-based network capable of generating high-quality speech from mel spectrograms. In its true sense, it is a generative model that generates audio by sampling from a distribution. To use a neural network as a generative model, we take samples from a simple distribution, in our case, a zero mean spherical Gaussian with the same number of dimensions as our desired output, and put those samples through a series of layers that transforms the simple distribution to one which has the desired distribution. In this case, we model the distribution of audio samples conditioned on a mel spectrogram.



**Step 1: Mounting Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Step 2: Installing dependencies required**

In [ ]:
!pip install hparams
!pip install unidecode

**Step 3: Migrating to the tacotron 2 folder and changing the tensorflow version to 1.xx**

In [ ]:
%cd "/content/drive/MyDrive/SSMT/tacotron2"
%tensorflow_version 1.x

/content/drive/MyDrive/SSMT/tacotron2
TensorFlow 1.x selected.


**Step 4: Importing the necessary packages and files needed to run the inference**

In [ ]:
import matplotlib
%matplotlib inline
import matplotlib.pylab as plt

import IPython.display as ipd

import sys
sys.path.append('waveglow/')
import numpy as np
import torch

from hparams import create_hparams
from model import Tacotron2
from layers import TacotronSTFT, STFT
from audio_processing import griffin_lim
from train import load_model
from text import text_to_sequence
from denoiser import Denoiser

**Step 5: Setting up the mel spectrogram plotting function and calling the hyperparamater generator in its default setting**

In [ ]:
# This function is used to plot the mel-spectrogram of the generated output and alignment graph
def plot_data(data, figsize=(16, 4)):
    fig, axes = plt.subplots(1, len(data), figsize=figsize)
    for i in range(len(data)):
        axes[i].imshow(data[i], aspect='auto', origin='bottom', 
                       interpolation='none')

In [ ]:
hparams = create_hparams()
hparams.sampling_rate = 22050

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



**Step 6: Load model from checkpoint**

In [ ]:
checkpoint_path = "/content/drive/MyDrive/SSMT/checkpoints/test12/checkpoint_4400"
model = load_model(hparams)
model.load_state_dict(torch.load(checkpoint_path)['state_dict'])
_ = model.cuda().eval().half()

**`Step 7: Load WaveGlow for mel2audio synthesis and denoiser`**

In [ ]:
waveglow_path = '/content/drive/MyDrive/SSMT/waveglow_256channels_universal_v5.pt'
waveglow = torch.load(waveglow_path)['model']
waveglow.cuda().eval().half()
for k in waveglow.convinv:
    k.float()
denoiser = Denoiser(waveglow)

**Step 8: Prepare text input**

In [ ]:
text = "मैं बाज़ार जाता हूँ "
sequence = np.array(text_to_sequence(text, ['transliteration_cleaners']))[None, :]
sequence = torch.autograd.Variable(
    torch.from_numpy(sequence)).cuda().long()

**Step 9: Decode text input and plot results**

In [ ]:
mel_outputs, mel_outputs_postnet, _, alignments = model.inference(sequence)
plot_data((mel_outputs_postnet.float().data.cpu().numpy()[0],
           alignments.float().data.cpu().numpy()[0].T))

***A short summary about melspectrograms:***

Sound is heard as a result of the variation of pressure with time. However, speech signals are complex entities and a simple pressure variation does not capture enough information for the deep learning model to be trained. Hence in short, a melspectrogram, is a graph which plots three quanitites - Time on the X axis, Frequency on the Y axis and the colors represent the loudness of the sound. 

The alignment graph seen above is a simple representation of the trajectory of the final output compared to its initial text input





**Step 10: Synthesize audio from spectrogram using WaveGlow**

In [ ]:
with torch.no_grad():
    audio = waveglow.infer(mel_outputs_postnet, sigma=0.666)
ipd.Audio(audio[0].data.cpu().numpy(), rate=hparams.sampling_rate)

**Step 11: (Optional) Remove WaveGlow bias**

In [ ]:
audio_denoised = denoiser(audio, strength=0.01)[:, 0]
ipd.Audio(audio_denoised.cpu().numpy(), rate=hparams.sampling_rate) 

**References**

* "Natural TTS Synthesis by Conditioning WaveNet on Mel Spectrogram Predictions" - Shen et al, ICASSP 2018

* "WaveGlow: A Flow-based Generative Network for Speech Synthesis" - Prenger et al, ICASSP 2019

* Indic TTS Databasee - IIT Madras : https://www.iitm.ac.in/donlab/tts/index.php

* Blog on English TTS Using Tacotron 2 and WaveGlow by Nvidia: https://developer.nvidia.com/blog/generate-natural-sounding-speech-from-text-in-real-time/


